In [1]:
import math
import folium
from folium import plugins

In [17]:
class lens:
    def __init__(self, label, focal_length, width, heigth, pixel_size):
        self.label = label
        self.focal_length = focal_length
        self.width = width
        self.heigth = heigth
        self.pixel_size = pixel_size

    def fov(self):
        self.h_fov = 2 * math.degrees(math.atan(self.width * self.pixel_size / (2 * self.focal_length)))
        self.v_fov = 2 * math.degrees(math.atan(self.heigth * self.pixel_size / (2 * self.focal_length)))

    def __repr__(self):
        return f"lens({self.label}, Focal_length:{self.focal_length}, Width:{self.width}, Heigth:{self.heigth}, Pixel Size:{self.pixel_size}, H-FoV:{self.h_fov}, V-FoV:{self.v_fov})"


In [18]:
lens_6_degrees = lens("6_degrees", 72.8, 640, 512, 0.012)
lens_8_degrees = lens("8_degrees", 55, 640, 512, 0.012)
lens_6_degrees.fov()
lens_8_degrees.fov()

In [19]:
lens_6_degrees, lens_8_degrees

(lens(6_degrees, Focal_length:72.8, Width:640, Heigth:512, Pixel Size:0.012, H-FoV:6.038793552433804, V-FoV:4.832644878467438),
 lens(8_degrees, Focal_length:55, Width:640, Heigth:512, Pixel Size:0.012, H-FoV:7.987612382993679, V-FoV:6.393815971102685))

In [20]:
# Ground Coverage=2×(altitude×tan(FoV/2))
altitude = 460
h_ground_coverage_6_degrees = round((2 * altitude * math.tan(math.radians(lens_6_degrees.h_fov / 2))), 2)
v_ground_coverage_6_degrees = round((2 * altitude * math.tan(math.radians(lens_6_degrees.v_fov / 2))), 2)
h_ground_coverage_8_degrees = round((2 * altitude * math.tan(math.radians(lens_8_degrees.h_fov / 2))), 2)
v_ground_coverage_8_degrees = round((2 * altitude * math.tan(math.radians(lens_8_degrees.v_fov / 2))), 2)

In [21]:
h_ground_coverage_6_degrees, v_ground_coverage_6_degrees, h_ground_coverage_8_degrees, v_ground_coverage_8_degrees

(48.53, 38.82, 64.23, 51.39)

In [22]:
# Define a function to calculate degrees per km for latitude and longitude
def km_to_degrees(lat_km, lon_km):
    # Approximation, 1 degree latitude ~ 111 km, 1 degree longitude ~ 111 km * cos(latitude)
    lat_deg = lat_km / 111.32  # Latitude degrees per km
    lon_deg = lon_km / (111.32 * math.cos(math.radians(0)))  # Longitude degrees per km at equator
    return lat_deg, lon_deg

In [23]:
# 40°49'35"N, 14°08'20"E (Campi Flegrei)
latitude, longitude = 40.82639, 14.13889

In [24]:
# Create the map centered at the given coordinates
m = folium.Map(location=[latitude, longitude], zoom_start=12)

In [25]:
# Draw the coverage for the 6° FOV (72 mm)
h_coverage_6 = h_ground_coverage_6_degrees
v_coverage_6 = v_ground_coverage_6_degrees

# Convert km to degrees
lat_deg_6, lon_deg_6 = km_to_degrees(v_coverage_6, h_coverage_6)

# Create a rectangle for 6° FOV
folium.Rectangle(
    bounds=[(latitude - lat_deg_6 / 2, longitude - lon_deg_6 / 2),
            (latitude + lat_deg_6 / 2, longitude + lon_deg_6 / 2)],
    color="blue", weight=2, fill=True, fill_opacity=0.3
).add_to(m)

In [26]:
lat_deg_6, lon_deg_6

(0.3487243981315128, 0.43595041322314054)

In [27]:
# Draw the coverage for the 8° FOV (55 mm)
h_coverage_8 = h_ground_coverage_8_degrees
v_coverage_8 = v_ground_coverage_8_degrees

# Convert km to degrees
lat_deg_8, lon_deg_8 = km_to_degrees(v_coverage_8, h_coverage_8)

# Create a rectangle for 8° FOV
folium.Rectangle(
    bounds=[(latitude - lat_deg_8 / 2, longitude - lon_deg_8 / 2),
            (latitude + lat_deg_8 / 2, longitude + lon_deg_8 / 2)],
    color="red", weight=2, fill=True, fill_opacity=0.3
).add_to(m)

In [28]:
folium.Marker([latitude, longitude], popup="Satellite Location").add_to(m)

In [29]:
folium.LayerControl().add_to(m)

In [30]:
m.save('satellite_coverage_map_centered_Mine.html')